# Predicción con modelo de regresión logística

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [28]:
df = pd.read_excel("../data/datos_churn_limpios.xlsx")

## Paso 1: Preparación de los datos

In [29]:
# Eliminar columnas irrelevantes
cols_eliminar = ["customer_id", "state", "contact_date", "last_transaction_date", "churn_category", "churn_reason"]
df = df.drop(columns=cols_eliminar, errors='ignore')

In [30]:
# Convertir churn_label a valores binarios (0 = No Churn, 1 = Churn)
df["churn_label"] = df["churn_label"].map({"no": 0, "yes": 1})

In [31]:
# Separar variables categóricas y numéricas
col_cat = df.select_dtypes(include=['object']).columns.tolist()
col_num = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
col_num.remove("churn_label")  # Excluir la variable objetivo

In [32]:
# Aplicar One-Hot Encoding a las variables categóricas. (Convierte variables categóricas en numéricas para que el modelo las entienda)
df = pd.get_dummies(df, columns=col_cat, drop_first=True)

In [33]:
# Escalar las variables numéricas. (Escala todas las variables numéricas para mejorar el rendimiento del modelo).
scaler = StandardScaler()
df[col_num] = scaler.fit_transform(df[col_num])

In [34]:
# Separar features (variables predictoras, X) y target (variable objetivo, Y)
X = df.drop(columns=["churn_label"])
y = df["churn_label"]

In [35]:
# Dividir los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,  # 20% para prueba, 80% para entrenamiento
                                                    random_state=42, # Fijar la semilla para reproducibilidad
                                                    stratify=y) # Mantener la proporción original de clases
# Si usamos todos los datos para entrenar y luego evaluamos en los mismos datos, no sabremos si el modelo realmente funciona en casos nuevos.

In [36]:
# Verificar la forma de los conjuntos de datos
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5352, 31), (1338, 31), (5352,), (1338,))

## Paso 2: Entrenar el modelo de Regresión Logística

In [ ]:
# Inicializar y entrenar el modelo de Regresión Logística
log_model = LogisticRegression(random_state=42, max_iter=1000)
log_model.fit(X_train, y_train)

In [38]:
# Hacer predicciones en el conjunto de prueba
y_pred = log_model.predict(X_test)

In [39]:
# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [40]:
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Visualizar la matriz de confusión
plt.figure(figsize=(5, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["No Churn", "Churn"], yticklabels=["No Churn", "Churn"])
plt.xlabel("Predicción")
plt.ylabel("Valor Real")
plt.title("Matriz de Confusión - Regresión Logística")
plt.show()

In [ ]:
# Reporte de clasificación
classification_rep = classification_report(y_test, y_pred, output_dict=True)

# Mostrar resultados
accuracy, precision, recall, f1, classification_rep

### Variables más importantes en la predicción del abandono

In [43]:
# Extraer coeficientes de la regresión logística
feature_importance = pd.DataFrame({"Feature": X.columns, "Importance": np.abs(log_model.coef_[0])})

In [44]:
# Ordenar por importancia
feature_importance = feature_importance.sort_values(by="Importance", ascending=False)

In [ ]:
# Visualizar los 10 factores más importantes
plt.figure(figsize=(10, 5))
sns.barplot(x="Importance", y="Feature", data=feature_importance.head(10), palette="viridis")
plt.title("Top 10 Variables más Importantes en la Predicción del Churn")
plt.xlabel("Importancia Absoluta")
plt.ylabel("Variables")
plt.show()

## Conclusiones del modelo predictivo

El análisis de los coeficientes del modelo de Regresión Logística nos ha permitido **identificar las variables que más influyen en la predicción del abandono de clientes**. Estos hallazgos son **clave para entender qué factores determinan el abandono y cómo podemos actuar para reducirlo**.  

### Tipo de contrato  
- Los clientes con contratos de dos años tienen menor probabilidad de abandonar.  
- Los clientes con contratos mensuales abandonan con mayor frecuencia.  
- Esto confirma que cuanto más largo es el contrato, más difícil es que un cliente se vaya.  
- **Acción recomendada:** Fomentar los contratos de largo plazo con incentivos como descuentos o beneficios exclusivos.  

### Número de llamadas al servicio al cliente  
- Cuantas más llamadas hace un cliente al soporte, mayor es la probabilidad de que abandone.  
- Esto indica que los clientes con problemas no están obteniendo una solución efectiva y terminan cancelando el servicio.  
- **Acción recomendada:** Mejorar la atención al cliente, asegurándose de resolver problemas en las primeras interacciones.  

### Pertenecer a un grupo familiar  
- Los clientes con un plan familiar tienen menor probabilidad de abandonar.  
- Esto confirma que compartir la cuenta con otras personas genera un mayor compromiso con el servicio.  
- **Acción recomendada:** Promocionar más los planes familiares con descuentos para fomentar la retención.  
